### 參考: https://medium.com/%E6%89%8B%E5%AF%AB%E7%AD%86%E8%A8%98/%E8%87%AA%E7%84%B6%E8%AA%9E%E8%A8%80%E8%99%95%E7%90%86-%E4%BD%BF%E7%94%A8-n-gram-%E5%AF%A6%E7%8F%BE%E8%BC%B8%E5%85%A5%E6%96%87%E5%AD%97%E9%A0%90%E6%B8%AC-10ac622aab7a

In [1]:
from collections import Counter, namedtuple
import json
import re

In [2]:
DATASET_DIR = './WebNews.json'
with open(DATASET_DIR, encoding = 'utf8') as f:
    dataset = json.load(f)

In [3]:
seg_list = list(map(lambda d: d['detailcontent'], dataset))
rule = re.compile(r"[^\u4e00-\u9fa5]")
seg_list = [rule.sub('', seg) for seg in seg_list]
seg_list

['桃園東門停車場都更年底前完工結合公益設施帶動舊市區發展桃園市長鄭文燦今日下午前往桃園區視察東門段公辦都更基地時表示為推動桃園舊城都市更新市府推動桃園區東門段東門停車場公辦都更工程在年月與立信工營造股份有限公司簽訂契約經歷年多都市更新擬定及審議過程於年月取得建造執照並於月開工總投資金額約為億元成為桃園區首件動工的都市更新案東門停車場公辦都更案將提供托嬰中心親子館及社會住宅等公益設施預計年底前完工完工後不僅可創造優質的都市環境也可帶動當地發展鄭市長指出桃園市東門停車場桃園區東門段地號等筆土地公辦都更案是桃園區都更案的第一步配合鐵路地下化及捷運綠線等計畫加上統領廣場改裝後來客數眾多站前商圈漸趨活絡因此市府致力推動站前都市更新優先選定桃園市東門停車場公辦都更案作為北桃園公辦都更示範基地鄭市長提到東門停車場都更案則規劃為棟地上層地下層的建築物擴增停車空間為汽車停車數輛機車停車數輛同時市府也預計可取得戶社會住宅優先讓警消人員居住亦提供托嬰中心親子館人行步道及開放空間期盼該都更案不僅作為桃園市區複合式住宅最好的典範也能讓市民朋友擁有更完善的活動空間鄭市長談及感謝立信工營造股份有限公司從簽約完成至開工僅花費個月相當不容易整體上除了團隊全力配合外市府也主動協助排除各項困難顯示市府主導推動都市更新的效率及決心此外位於正光路警察宿舍公辦都更案也進行公開招標預估完工後可分回戶社會住宅將優先提供警消宿舍使用同時復興路及民生路路口亦有復興路公辦都更案也正在招商中期盼以都更案帶動地方發展提高生活品質鄭市長也說桃園舊市區具備不少優點蘊含人情味與市民的回憶伴隨桃園三鐵共構的發展各項捷運建設計畫陸續進行未來桃園新站特區會愈具特色後續也請都發局進行策略性的都更提高基礎容積及獎勵容積帶動都更的動力進而讓桃園車站及舊市區產生新功能今日包括市議員李光達余信憲陳美梅黃家齊簡智翔市府都發局長盧維屏財政局長歐美鐶消防局長謝呂泉社會局長鄭貴華住宅發展處長莊敬權桃園區長陳玉明等均一同進行視察',
 '三倍券今起開放領用桃市加碼優惠獎不完振興三倍券今日正式開放領用桃園市警察局昨日調派警力協助護運三倍券至本市家超商近期也將針對郵局超商等營業處所加強安全維護工作桃園市副市長李憲明上午主持防疫暨紓困振興會議時表示本市配合中央推動振興紓困方案並積極推出加碼優惠方案希望刺激經濟活絡發展經發局長郭裕信表示截至月日晚間全國超過萬人

In [4]:
def ngram(documents, N=2):
    ngram_prediction = dict()
    total_grams = list()
    words = list()
    Word = namedtuple('Word', ['word', 'prob'])

    for doc in documents:
        split_words = ['<s>'] + list(doc) + ['</s>']
        # 計算分子
        [total_grams.append(tuple(split_words[i:i+N])) for i in range(len(split_words)-N+1)]
        # 計算分母
        [words.append(tuple(split_words[i:i+N-1])) for i in range(len(split_words)-N+2)]
        
    total_word_counter = Counter(total_grams)
    word_counter = Counter(words)
    
    for key in total_word_counter:
        word = ''.join(key[:N-1])
        if word not in ngram_prediction:
            ngram_prediction.update({word: set()})
            
        next_word_prob = total_word_counter[key]/word_counter[key[:N-1]]
        w = Word(key[-1], '{:.3g}'.format(next_word_prob))
        ngram_prediction[word].add(w)
        
    return ngram_prediction

In [5]:

tri_prediction = ngram(seg_list, N=3)
for word, ng in tri_prediction.items():
    tri_prediction[word] = sorted(ng, key=lambda x: x.prob, reverse=True)

In [6]:
text = '韓國'
next_words = list(tri_prediction[text])[:5]
for next_word in next_words:
    print('next word: {}, probability: {}'.format(next_word.word, next_word.prob))

next word: 及, probability: 0.231
next word: 現, probability: 0.154
next word: 模, probability: 0.0769
next word: 雙, probability: 0.0769
next word: 新, probability: 0.0769
